# Algorithm 5 (Exploration by Confidence)

In [201]:
import pandas as pd
import math 

In [202]:
# Достоверность
c = 0.5
context_df_expert = pd.read_csv('example_expert.csv',sep=';')
context_expert = (context_df_expert.values.tolist(),context_df_expert.shape)

In [203]:
context_df = pd.read_csv('example.csv',sep=';')

In [204]:
context_df

,hair,feathers,eggs,milk,predator,breathes
0,0,0,1,0,1,0
1,1,0,0,1,1,0
2,1,0,0,1,1,1
3,1,0,0,1,0,1
4,1,0,0,1,0,1
5,0,0,1,0,0,0
6,0,0,1,0,1,0
7,1,0,0,1,0,1
8,1,0,0,1,1,1
9,0,1,1,0,0,1


In [205]:
context = (context_df.values.tolist(),context_df.shape)
context[1][0]

11

In [206]:
context_Li = ([],(0,6))
context_Li[1] = (context_Li[1][0], context_Li[1][1])

TypeError: 'tuple' object does not support item assignment

In [127]:
context_Li

([], (0, 6))

In [102]:
class Implication:
    def __init__(self,p,c):
        self.premise = p.copy()
        self.consequent = c.copy()
        pass
    def __str__(self):
        return str(self.premise)+'->'+str(self.consequent)
    def __repr__(self):
        return str(self.premise)+'->'+str(self.consequent)
    def print_for_context(self, cont):
        old_values = range(0, len(cont))
        p_new = list(map(dict(zip(old_values, cont)).get, self.premise))
        c_new = list(map(dict(zip(old_values, cont)).get, self.consequent))
        return str(p_new)+'->'+str(c_new)

In [103]:
def Galois(X,context,t):
    if len(X)==0:
        if t=='ObjectsToAttributes': return set(range(0, context[1][1]))
        if t=='AttributesToObjects': return set(range(0, context[1][0]))
    if t=='ObjectsToAttributes':
        context_temp=[context[0][x] for x in list(X)]
    if t=='AttributesToObjects':
        transposed=list(zip(*context[0]))
        context_temp=[transposed[x] for x in list(X)]
    at=[sum(i) for i in zip(*context_temp)]
    attributes=set()
    for i in range(0,len(at)):
        if at[i]==len(X):   
            attributes.update(set([i]))
    return attributes

In [104]:
Galois({1, 2, 3, 4, 7, 8, 9, 10},context,'ObjectsToAttributes')

{5}

In [105]:
A = {3}
A = Galois(A,context,'AttributesToObjects')
A = Galois(A,context,'ObjectsToAttributes')
A

{0, 3, 5}

In [106]:
def ClosureOnImp(A,L):
    Li=L.copy()
    while True:
        #print('Closure Start')
        stable=True   
        Lj=Li.copy()
        for i in Lj:
            if (i.premise.issubset(A)):
                A.update(i.consequent)
                stable=False
                Li.difference_update(set([i]))
        if stable==True:
            break
    return A

In [107]:
A = {5}
L = {Implication({0}, {3}), #hair -> milk
              Implication({0}, {4}), #hair -> predator
              Implication({0}, {5}), #hair -> breathes
              Implication({1}, {2}), #feathers -> eggs
              Implication({1}, {4}), #feathers -> predator
              Implication({1}, {5}), #feathers -> breathes
              Implication({2}, {4}), #eggs -> predator
              Implication({3}, {0}), #milk -> hair
              Implication({3}, {4}), #milk -> predator
              Implication({3}, {5}), #milk -> breathes
              Implication({4}, {0}), #predator -> hair
              Implication({4}, {2}), #predator -> eggs
              Implication({4}, {3}), #predator -> milk
              Implication({4}, {5}), #predator -> breathes
              Implication({5}, {0}), #breathes -> hair
              Implication({5}, {3}), #breathes -> milk
              Implication({5}, {4}), #breathes -> predator
              }


In [108]:
ClosureOnImp(A,L)

{0, 2, 3, 4, 5}

In [109]:
def ClosureOnContext(A,context_):
    A = Galois(A,context_,'AttributesToObjects')
    A = Galois(A,context_,'ObjectsToAttributes')
    return A

In [141]:
def SmallestSet (A,M,Ki,context_KLi):
    for m in list(reversed(M)):
        if m in A:
            A.difference_update(set([m]))
        else:           
            A_temp=A.copy()
            A_temp.update(set([m]))
            B=ClosureOnImp(A_temp,Ki)
            B_diff=B.copy()
            B_diff.difference_update(A)  
            B_L=B.copy()
            if all(i>=m for i in B_diff) and (B != ClosureOnContext(B_L, context_KLi)):
                return B
    return -1

In [142]:
A = set()
context_K = (context_df.values.tolist(),context_df.shape)
M=list(range(0,context_K[1][1]))
Th_c_K = {Implication({0}, {3}), #hair -> milk
              Implication({0}, {4}), #hair -> predator
              Implication({0}, {5}), #hair -> breathes
            #   Implication({0}, {6}), #hair -> mammal
              Implication({1}, {2}), #feathers -> eggs
              Implication({1}, {4}), #feathers -> predator
              Implication({1}, {5}), #feathers -> breathes
            #   Implication({1}, {8}), #feathers -> bird
              Implication({2}, {4}), #eggs -> predator
            #   Implication({2}, {7}), #eggs -> fish
              Implication({3}, {0}), #milk -> hair
              Implication({3}, {4}), #milk -> predator
              Implication({3}, {5}), #milk -> breathes
            #   Implication({3}, {0}), #milk -> mammal
              Implication({4}, {0}), #predator -> hair
              Implication({4}, {2}), #predator -> eggs
              Implication({4}, {3}), #predator -> milk
              Implication({4}, {5}), #predator -> breathes
            #   Implication({4}, {6}), #predator -> mammal
              Implication({5}, {0}), #breathes -> hair
              Implication({5}, {3}), #breathes -> milk
              Implication({5}, {4}), #breathes -> predator
            #   Implication({5}, {6}), #breathes -> mammal
            #   Implication({6}, {0}), #mammal -> hair
            #   Implication({6}, {3}), #mammal -> milk
            #   Implication({6}, {4}), #mammal -> predator
            #   Implication({6}, {5}), #mammal -> breathes
            #   Implication({7}, {2}), #fish -> eggs
            #   Implication({7}, {4}), #fish -> predator
            #   Implication({8}, {1}), #bird -> feathers
            #   Implication({8}, {2}), #bird -> eggs
            #   Implication({8}, {4}), #bird -> predator
            #   Implication({8}, {5}), #bird -> breathes
              }
Ki = {Implication({1}, {2})}

In [143]:
context_KLi = context
SmallestSet (A,M,Ki,context_K)

{3}

In [113]:
def FindConfidence(imp, context):
    galois_PC = Galois(set.union(imp.premise, imp.consequent), context, 'AttributesToObjects')
    galois_C = Galois(imp.consequent, context, 'AttributesToObjects')
    return len(galois_PC)/len(galois_C)

In [114]:
def SmallestIntent (P, M, context_KLi, context_Li, Ki, context_K, m):
    for m in list(reversed(M)):
        if m in P:
            P.difference_update(set([m]))
        else:
            P_temp=P.copy()
            P_temp.update(set([m]))
            B=ClosureOnContext(P_temp,context_KLi)
            B_diff=B.copy()
            B_diff.difference_update(P)  

            B_L=B.copy()
            if (context_Li == []):
                B_L = set()
            else:
                ClosureOnContext(B_L, context_Li)

            B_K = B.copy()
            ClosureOnImp(B_K, Ki)

            B_L_K_diff = B_L.difference(B_K)

            exist = False

            for m in B_L_K_diff:
                if FindConfidence(Implication(B, {m}), context_K) >= c :
                    exist = True
                    break

            if all(i>=m for i in B_diff) and exist:
                return B
    return -1

In [175]:
context_K = [context_df.values.tolist(),context_df.shape]
context_K

[[[0, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 1, 1],
  [0, 1, 1, 0, 0, 1],
  [0, 1, 1, 0, 1, 1]],
 (11, 6)]

In [182]:
context_Li = [[[1, 0, 0, 1, 0, 1]],(1,6)]

In [183]:
import copy
context_KLi = copy.deepcopy(context_K)
context_KLi[0].append(context_Li[0][0])

In [189]:
context_KLi[0][0]

[0, 0, 1, 0, 1, 0]

In [185]:
context_Li

[[[1, 0, 0, 1, 0, 1]], (1, 6)]

In [186]:
Ki

{{0}->{3}}

In [187]:
context_K

[[[0, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 1, 0, 1, 0],
  [1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 1, 1],
  [0, 1, 1, 0, 0, 1],
  [0, 1, 1, 0, 1, 1]],
 (11, 6)]

In [207]:
context_Li = ([[1, 0, 0, 1, 1, 1]],(1,6))
SmallestIntent ({3}, M, context_KLi, context_Li, Ki, context_K, -1)

-1

In [115]:
def expert_p(imp, context):
    AI =  Galois(imp.premise, context, 'AttributesToObjects')
    BI = Galois(imp.consequent, context, 'AttributesToObjects')
    # Является ли AI подмножеством BI
    if (AI.issubset(BI)) :
        return False
    else :
        # Контрпример
        counterexample = [0 for i in range(0, len(M))]
        for i in imp.premise :
            counterexample[i] = 1
        return counterexample

In [116]:
def col_num_to_name(col_nums, columns):
    old_values = range(0, len(columns))
    new = list(map(dict(zip(old_values, columns)).get, col_nums))
    return new

def example_to_column_names(example, columns):
    new = [columns[i] for i in range(0, len(example)) if example[i]==1]
    return new

def example_to_column_nums(example):
    new = [i for i in range(0, len(example)) if example[i]==1]
    return new

In [117]:
def DeleteFalseImplications(L, C) :
    ex = example_to_column_nums(C)
    L_updated = L.copy()
    for i in L :
        if not (not i.premise.issubset(ex) or i.consequent.issubset(ex)) :
            L_updated.remove(i)
    return L_updated

In [118]:
def LecticalComparison(P1, P2):
    i = 0
    while (P1[i] == P2[i]):
        i = i+1
    if (P1[i] > P2[i]):
        return True
    else:
        return False

In [119]:
def ExploringFaultyData(K, Th_c_K, M, context_K, context_expert):
    print('Значение M - ' + str(M))
    print('Значение K - ' + str(K))
    print('Значение Th_c_K - ' + str(Th_c_K))
    # i
    i = 0
    Pi = ClosureOnImp(set(), K)
    print('Значение Pi - ' + str(Pi))
    Ki = K
    Li = Th_c_K
    context_Li = []
    print('Контекст Li - ' + str(context_Li))

    while (True):
        # ii
        context_KLi = context_K
        for c in context_Li:
            context_KLi[0].append(c)
        m = -1
        Pi_1 = SmallestIntent(Pi, M, context_KLi, context_Li, Ki, context_K, m)
        Pi_2 = SmallestSet(Pi, M, Ki, context_KLi)

        print('Pi_1 - ' + str(Pi_1))
        print('Pi_2 - ' + str(Pi_2))

        if Pi_1==-1 and Pi_2==-1:
            break
        if Pi_1==-1:
            Pi = Pi_2
        else:
            if Pi_2==-1:
                Pi = Pi_1
            else:
                if (LecticalComparison(Pi_1, Pi_2)):
                    Pi = Pi_1
                else:
                    Pi = Pi_2
        
        # Выбор Q
        if (Pi == Pi_1):
            Qi = Pi_1.add(m)
        else:
            Qi = ClosureOnContext(Pi_2, context_KLi)
        print('Выбранное Q - ' + str(Qi))

        # Инициализация импликации
        imp = Implication(Pi, Qi)
        print('Есть ли контрпример для ' + imp.print_for_context(context_df.columns))
        # Запрос к эксперту
        counterexample = expert_p(imp)
        print(counterexample)

        # iv - v
        if (counterexample) :
            # Удаление импликаций, противоречащих контрпримеру
            Li = DeleteFalseImplications(Li, counterexample)
            # Добавлен контрпример
            context_Li.append(counterexample)
            print('Контекст Li - ' + str(context_Li))
            print('Значение Li - ' + str(Li))
        else :
            Ki.add(imp)
            print(Ki)
        # vi
        i = i+1
    print(Ki)
    print(Li)

In [120]:
M=list(range(0,context[1][1]))
Th_c_K = {Implication({0}, {2}), Implication({1}, {2}), Implication({2}, {3}), Implication({2}, {3}), Implication({2}, {4}), Implication({4}, {5}), Implication({5}, {4, 3, 2}), Implication({4}, {3}), Implication({4}, {2}),  Implication({3}, {2})}
K = {Implication({4}, {2})}

In [ ]:
ExploringFaultyData(K, Th_c_K, M, context, context_expert)

Значение M - [0, 1, 2, 3, 4, 5]
Значение K - {{4}->{2}}
Значение Th_c_K - {{4}->{5}, {2}->{3}, {5}->{2, 3, 4}, {4}->{2}, {3}->{2}, {1}->{2}, {0}->{2}, {4}->{3}, {2}->{3}, {2}->{4}}
Значение Pi - set()
Контекст Li - []
Pi_1 - -1
Pi_2 - {3}
Выбранное Q - {0, 3, 5}
Есть ли контрпример для ['milk']->['hair', 'milk', 'breathes']


TypeError: expert_p() missing 1 required positional argument: 'context'